In [ ]:
import socks
import socket
from bs4 import BeautifulSoup

In [ ]:
def create_connection(address, timeout=None, source_address=None):
    sock = socks.socksocket()
    sock.connect(address)
    return sock

socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "127.0.0.1", 9050)

# patch the socket module
socket.socket = socks.socksocket
socket.create_connection = create_connection

In [ ]:
import urllib2
import requests
import pandas as pd

In [ ]:
scraped_posts = []
for page in range(1, 1000):
    try:
        response = urllib2.urlopen('http://npdaaf3s3f2xrmlo.onion/public?page='+str(page))
        soup = BeautifulSoup(response.read(), 'lxml')
        notes = soup.find("div", {"id": "note_list"})

        for child in notes.children:
            try:
                #for x in child.children:
                    #print x
                    #print "################"
                post_id = child.attrs['id']
                post_user = child.find("div", {"class": "user_note"}).text
                post_time = child.find("div", {"class": "date_note"}).attrs['data']
                post_content = child.find("div", {"class": "text_note"}).text

                scraped_posts.append({'id' : int(post_id.replace("status_", '')), 
                                      'user' : post_user.replace('\n', '').strip(), 
                                      'timestamp' : int(post_time.replace("{", '').replace("}", '').replace('time:','')), 
                                      'content' : post_content.replace('\n', '').strip()})

            except Exception:
                continue
    except Exception:
        print "Error on page", page
        continue

In [ ]:
df_posts = pd.DataFrame(scraped_posts)

In [ ]:
df_posts.to_csv('to_1000.csv', encoding='utf-8')